In [3]:
import numpy as np
import pandas as pd
import itertools as it
import pickle
import glob
import os
import string
import gc
import re
import time
import nltk
import spacy
import textacy
#import en_core_web_md
import sematch
import gensim

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from tqdm import tqdm, tqdm_notebook
from scipy import sparse
from scipy.optimize import minimize
    
from clean_utils import *
from parser_utils import *
%load_ext autoreload
%autoreload 2

In [28]:
from twit_prepocess import tokenize

In [45]:
# text = "I TEST alllll kinds of #hashtags and #HASHTAGS, @mentions and 3000 (http://t.co/dkfjkdf). w/ <3 :) haha!!!!!"
tokenize(data.comment_text[6])

'cocksucker <allcaps> before <allcaps> you <allcaps> piss <allcaps> around <allcaps> on <allcaps> my <allcaps> work <allcaps>'

In [44]:
data.comment_text[6]

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
def word_stat(word):
    dlst = len(word)
    n_punc = len([i for i in word if i in string.punctuation])
    n_upper = len([i for i in word if i.isupper()])
    return np.array([dlst, n_punc, n_upper])

In [ ]:
comment=comment.lower()
comment=re.sub("\\n","",comment)
#comment = myDate.sub(' xxDATExx ', comment)
comment=re.sub("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}","",comment)
    comment=re.sub("\[\[.*\]","",comment)
    words=tokenizer.tokenize(comment)
    words=[APPO[word] if word in APPO else word for word in words]
    words=[lem.lemmatize(word, "v") for word in words]
    words = [w for w in words if not w in eng_stopwords]   
    clean_sent=" ".join(words)

In [2]:
def clean_lemmat_quora(df, shapes):
    t = time.time()
    print('Cleaning based on forums functions.', df.shape)
    df = df.apply(lambda x :clean(x))
    print('Cleaning using textacy.', df.shape)
    df = df.apply(lambda x: textacy.preprocess.preprocess_text(x, fix_unicode = True,
                                                           no_urls=True, no_emails=True, 
                                                               no_phone_numbers=True, no_numbers=True,
                                                                            lowercase = True,
                                                                            no_contractions = True,
                                                                            transliterate = True))
    print('Lemmatizing text.', df.shape)
    SYMBOLS = set(' '.join(string.punctuation).split(' ') + ['...', '“', '”', '\'ve'])
    q1 = []
    c = 0
    for doc in tqdm(nlp.pipe(df, n_threads=8, batch_size=10000)):
        c+=1
        word_list = ([c.lemma_ for c in doc if c.lemma_ not in SYMBOLS])
        q1.append(' '.join(i for i in word_list))
    q1 = pd.DataFrame(q1, columns=['comment_text'])
    df_train_lemm = q1.iloc[:shapes[0]]
    df_test_lemm = q1.iloc[shapes[0]:]
    print('Correcting words. Using most probable substitutes.', q1.shape)
    q1.comment_text = q1.comment_text.apply(lambda x: (' '.join([correction(i) for i in word_tokenize(x)])))
    df_train_correct = q1.iloc[:shapes[0]]
    df_test_correct = q1.iloc[shapes[0]:]
    print('Text cleaning done, time it took:', time.time() - t, q1.shape)
    return df_train_lemm, df_test_lemm, df_train_correct, df_test_correct

In [1]:
NROWS = 10

In [4]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id', nrows=NROWS)
shapes = data.shape
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id', nrows=NROWS)], axis=0)
data.train.fillna(0, inplace=True)

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('vocs/crawl-300d-2M.vec')

words = model.index2word

%%time
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

In [5]:
import joblib

In [6]:
joblib.dump(WORDS, 'vocs/words_for_check_spell.pkl')

['vocs/words_for_check_spell.pkl']

In [6]:
WORDS = joblib.load('vocs/words_for_check_spell.pkl')

In [7]:
nlp = spacy.load('en')

In [8]:
df_train_lemm, df_test_lemm, df_train_corr, df_test_corr = clean_lemmat_quora(data.comment_text, shapes)

Cleaning based on forums functions. (312735,)
Cleaning using textacy. (312735,)


0it [00:00, ?it/s]

Lemmatizing text. (312735,)


312735it [48:25, 107.65it/s]


Correcting words. Using most probable substitutes. (312735, 1)
Text cleaning done, time it took: 3450.4344255924225 (312735, 1)


In [12]:
df_train_corr.shape, df_test_corr.shape, df_train_lemm.shape, df_test_lemm.shape

((159571, 1), (153164, 1), (159571, 1), (153164, 1))

In [28]:
df_train_corr.to_csv('pre_data/df_train_spacycorr.csv', index = True, sep=';', index_label='index')
df_test_corr.to_csv('pre_data/df_test_spacycorr.csv', index = True, sep=';', index_label='index')
df_train_lemm.to_csv('pre_data/df_train_spacylemm.csv', index = True, sep=';', index_label='index')
df_test_lemm.to_csv('pre_data/df_test_spacylemm.csv', index = True, sep=';', index_label='index')

In [30]:
train = pd.read_csv('pre_data/df_train_spacycorr.csv', sep=';')
print(train.shape)
train.head()

(159571, 2)


,index,comment_text
0,0,explanationwhy edit make username hardcore met...
1,1,d'aww match background colour i be seemingly s...
2,2,hey man i be really try edit war -PRON- be guy...
3,3,morei can not make real suggestion improvement...
4,4,sir hero chance remember page that be
